# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [ ]:
import pandas as pd

def cargarDatos():
    anchoColumnas = [(0, 10), (10, 40), (40, 50), (50, 55)]
    nombresColumnas = ['fecha', 'producto', 'precio', 'cantidad']
    
    try:
        datos = pd.read_fwf(
            'datos.dat',
            colspecs=anchoColumnas,
            names=nombresColumnas,
            dtype={'producto': str, 'precio': float, 'cantidad': int}
        )

        datos['producto'] = datos['producto'].apply(lambda x: x.strip())
        datos['fecha'] = pd.to_datetime(datos['fecha'], errors='coerce')

        if datos.isna().sum().sum() > 0:
            print("aviso: Existen valores nulos en los datos cargados.")

        return datos

    except FileNotFoundError as error:
        print("error: Archivo 'datos.dat' no fua encontrado.")
        return pd.DataFrame(columns=nombresColumnas)

    except Exception as error:
        print(f"error durante la carga de datos: {error}")
        return pd.DataFrame(columns=nombresColumnas)

datos = cargarDatos()
datos


### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [ ]:
def calcularTotales(datos):
    if datos.shape[0] == 0:
        return 0, 0

    totalImporte = sum(datos['precio'] * datos['cantidad'])
    totalCantidad = datos['cantidad'].agg('sum')
    
    return totalImporte, totalCantidad

### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [ ]:
def unidadesVendidas(datos):
    if datos.shape[0] == 0:
        return pd.Series(dtype='int64')

    totalUnidades = datos.groupby('producto')['cantidad'].aggregate('sum')
    
    return totalUnidades.sort_values(ascending=False)

###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [ ]:
def precioPromedio(datos):
    if datos.shape[0] == 0:
        return pd.Series(dtype='float64')

    promedioPrecio = datos.groupby('producto')['precio'].mean().round(2)
    
    return promedioPrecio

### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [ ]:
def rankingProductos(datos, top=3):
    if datos.shape[0] == 0:
        return pd.Series(dtype='int64')

    topVentas = datos.groupby('producto')['cantidad'].aggregate('sum')
    
    return topVentas.nlargest(top)


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [ ]:
def ventasPorMes(datos):
    if datos.shape[0] == 0:
        return pd.DataFrame()

    datos = datos.assign(mes=datos['fecha'].dt.to_period('M'))
    
    ventasMensuales = datos.pivot_table(
        values='cantidad',
        index='mes',
        columns='producto',
        aggfunc='sum',
        fill_value=0
    )

    return ventasMensuales

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [ ]:
def resumenVentas(datos):

    if datos.shape[0] == 0:
        return pd.DataFrame(columns=['precioPromedio', 'unidadesVendidas', 'importeTotal'])
    
    ventasAgrupadas = datos.groupby('producto').agg(
        unidadesVendidas=('cantidad', 'sum'),
        importeTotal=('precio', lambda p: (p * datos.loc[p.index, 'cantidad']).sum())
    )
    
    ventasAgrupadas['precioPromedio'] = (ventasAgrupadas['importeTotal'] / 
                                           ventasAgrupadas['unidadesVendidas']).round(2)
    
    return ventasAgrupadas[['precioPromedio', 'unidadesVendidas', 'importeTotal']].sort_index()

## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [4]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

A = np.vstack([X**2, X, np.ones(len(X))]).T

coeficientes, _, _, _ = np.linalg.lstsq(A, Y, rcond=None)

print("coeficiente a, b, c:", coeficientes)

Ypred = f(X, coeficientes)
print("valores dichos por la función", Ypred)

error = np.abs(Y - Ypred)
print("error en cada punto:", error)

tolerancia = 1e-10
if np.all(error < tolerancia):
    print("la funcion pasa exactamente por todos los puntos dados")
else:
    print("la funcion no pasa exactamente por todos los puntos dados")


(-2, 10, 0)

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [121]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x**2 + b * x + c

def error(y, ypred):
    return np.sum((y - ypred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscarCoeficientes():
    coeficientes = randint(-10, 10, 3)
    Ypred = f(X, coeficientes)
    errorActual = error(Y, Ypred)
    
    aprendizaje = 0.001
    
    while errorActual > 1:
        nuevosCoeficientes = coeficientes + np.random.uniform(-aprendizaje, aprendizaje, 3)
        
        yPredNuevo = f(X, nuevosCoeficientes)
        nuevoError = error(Y, yPredNuevo)
        
        if nuevoError < errorActual:
            coeficientes = nuevosCoeficientes
            errorActual = nuevoError
    
    return coeficientes

coeficientes = buscarCoeficientes()
print("Coeficientes encontrados:", coeficientes)

yPredFinal = f(X, coeficientes)
errorFinal = error(Y, yPredFinal)
print("error final", errorFinal)

Los coeficientes son [-1.781  8.961  0.615] y el error es 0.9958049999998968


array([-1.781,  8.961,  0.615])